In [0]:
%pip install nbformat databricks-sdk[openai]==0.38.0 dspy --quiet
dbutils.library.restartPython()

In [0]:
%run ../prompts/sql_prompts_with_DSPy 

In [0]:
%run ../prompts/ml_prompts_with_DSPy

In [0]:
%run ../prompts/optimization_prompts_with_DSPy

In [0]:
%run ../helper/GradingModule

In [0]:
module = Module(databricks.sdk.WorkspaceClient())

In [0]:
candidate_dict = {
  "X": {
    "SQL": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-sql/01-SQL-X",
    "Optimization": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-optimization/Optimization-X",
    "ML": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-ml/04-Machine-Learning-X"
  },
  "Y": {
    "SQL": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-sql/01-SQL-Y"
  },
  "Z": {
    "SQL": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-sql/01-SQL-Z"
  },
  "A": {
    "SQL": "/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-sql/01-SQL-A"
  }

}

human_graded_dict = {
  "X": {
    "SQL": [15,15,15,15,15,25],
    "Optimization": [1,0,1,1,1,1,1,0],
    "ML": [5, 30, 25, 5, 0]
  }
}

module_list = {
  "SQL": module_dict_sql,
  "Optimization": module_dict_optimization,
  "ML": module_dict_ml
}

llm_models = {
  'llm-405B': 'databricks/databricks-meta-llama-3-1-405b-instruct',
  # 'llm-70B': 'databricks/databricks-meta-llama-3-3-70b-instruct'
}

In [0]:
import time 

model_dict = {}

for k,v in llm_models.items():
  start_time = time.time()
  lm = dspy.LM(llm_models[k])
  dspy.settings.configure(lm=lm)
  answers_dict = {}
  for candidate, values in candidate_dict.items():
    for section, context_path in values.items():
      module.set_module_dict(module_list[section])
      results = module.get_error_and_answer_dict(context_path)
      results['answers_dict']['candidate'] = candidate
      results['answers_dict']['section'] = section
      results['answers_dict']['context'] = results['context'] 
      answers_dict[section] = results['answers_dict']
      
    end_time = time.time()
    execution_time = end_time - start_time
    print(f" time taken by model {k} for execution is {execution_time} seconds")
    model_dict[k] = {'answers_dict': answers_dict, 'execution_time': execution_time, 'lm':lm}
    df_to_write = spark.createDataFrame(pd.concat(model_dict['llm-405B']['answers_dict'].values(), ignore_index=True))
    df_to_write.write.mode("append").option("mergeSchema", "true").saveAsTable("users.abhay_jalisatgi.training_set")

In [0]:
pd.concat(model_dict['llm-405B']['answers_dict'].values(), ignore_index=True).display()

In [0]:
df_to_write = spark.createDataFrame(pd.concat(model_dict['llm-405B']['answers_dict'].values(), ignore_index=True))
df_to_write.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("users.abhay_jalisatgi.training_set")

In [0]:
# Use the code snippet below to check run history and cost per run etc 
# model_dict['llm-70B']['lm'].history[-1]